**Based on Zaccone: Python Parallel Programming Cookbook** 

### Threads using the with statement 

with the help of the **with** statement, you can allocate and release some resource exactly where you need it; for 
this reason, the with statement is called a **context manager**. 

In the threading module, all the objects provided by the **acquire()** and **release()** methods may be used in a with 
statement block.

In tensorflow, this is how we can replace the close() method with context managers for a session:

\# Using the `close()` method. <br>
sess = tf.Session() <br>
sess.run(...) <br>
sess.close() <br>

\# Using the `context manager`. <br>
with tf.Session() as sess: <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; sess.run(...) <br>

The following objects can be used as context managers for a with statement:
* Lock
* RLock
* Condition
* Semaphore

This example shows the basic use of the with statement. We have a set with the most 
important synchronization primitives. So, we test them by calling each one with the with 
statement:

In [1]:
import threading
import logging

In [2]:
#This embeds the thread name in every log message using the formatter code's %(threadName)s statement. 
#The logging module is thread-safe, so the messages from different threads are kept distinct in the output.

logging.basicConfig(level=logging.DEBUG, format='(%(threadName)-10s) %(message)s',)

In [3]:
def threading_with(statement):
    # We don't need to use statement.acquire() and then statement.release()
    # the with statement will automatically do it for us.
    with statement:
        logging.debug('%s acquired via with'  %statement)


In [4]:
def threading_not_with(statement):
    statement.acquire()
    try:
        logging.debug('%s acquired directly' %statement )
    finally:
        statement.release()


In [5]:
#let's create a test battery
lock = threading.Lock()
rlock = threading.RLock()
condition = threading.Condition()
mutex = threading.Semaphore(1)
threading_synchronization_list = [lock ,rlock , condition , mutex]


In [6]:
#in the for cycle we call the threading_with e threading_no_with function
for statement in threading_synchronization_list :
    t1 = threading.Thread(target=threading_with, args=(statement,))
    t2 = threading.Thread(target=threading_not_with, args=(statement,))
    t1.start()
    t2.start()
    t1.join()
    t2.join()


(Thread-4  ) <locked _thread.lock object at 0x7f4af82c3288> acquired via with
(Thread-5  ) <locked _thread.lock object at 0x7f4af82c3288> acquired directly
(Thread-6  ) <locked _thread.RLock object owner=139959621117696 count=1 at 0x7f4af8b1a5a0> acquired via with
(Thread-7  ) <locked _thread.RLock object owner=139959629510400 count=1 at 0x7f4af8b1a5a0> acquired directly
(Thread-8  ) <Condition(<locked _thread.RLock object owner=139959629510400 count=1 at 0x7f4af8b1a510>, 0)> acquired via with
(Thread-9  ) <Condition(<locked _thread.RLock object owner=139959621117696 count=1 at 0x7f4af8b1a510>, 0)> acquired directly
(Thread-10 ) <threading.Semaphore object at 0x7f4af82c4160> acquired via with
(Thread-11 ) <threading.Semaphore object at 0x7f4af82c4160> acquired directly
